Парсер Zoon.ru

Часть 1. Собирает ссылки на страницы нужных объектов с карты.

- название
- координаты
- адрес

Парсер написан 2021-07-06, изменен 2021-08-05.

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument('--proxy-server=212.192.198.116:57033') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

Собираю список ссылок для "перемещения" по карте.

Zoon отдает только 240 ссылок, поэтому чтобы вытащить все, нужно премещать центральную точку на карте.

Как устроена ссылка Zoon:

https://zoon.ru/msk/beauty/ — домен / город / категория

?search_query_form=1&m%5B5d676e3f382c1302a5273135%5D=1 — запрос

&center%5B%5D=55.74096935251004 — широта

&center%5B%5D=37.60026722540671 — долгота

&zoom=12 — масштаб

In [ ]:
# текущая ссылка на карту (пока разбиваю вручную)

driver.current_url

Составляю список "входных" урлов. От точки на северо-западе сначала "двигаюсь" на восток, затем возвращаюсь к начальной, смещаю точку на юг и снова на восток.

Степень подробности (масштаб) выбираю в зависимости от объекта. Для стоматологий хватает и 14 и 840 ссылок.

Для салонов красоты нужно - 16 и 3168 ссылок. И, видимо, два раза. Т.е. второй раз нужно "сместить" точку и пройти еще раз, чтобы выцепить все.

In [ ]:
%%time

# список для ссылок
url_list = []

latitude_a = 56.043347 # задаю начальную широту
longitude_a = 37.067043 # задаю начальную долготу

# компоненты ссылки
#url_part_1 ='https://zoon.ru/msk/beauty/?search_query_form=1&center%5B%5D='
url_part_1 ='https://zoon.ru/msk/medical/?search_query_form=1&query%5B%5D=Стоматологии&query_type=base&center%5B%5D='
url_part_2 = '&center%5B%5D='
url_part_3 = '&zoom=15' # последнее число масштаб карты

for y in range(66): # цикл для перемещения по широте
    
    url = url_part_1 + str(latitude_a) + url_part_2 + str(longitude_a) + url_part_3
    url_list += [url]
        
    for x in range(47): # цикл для перемещения по долготе
        longitude_a += 0.023332 # перемещаю точку по долготе на восток
        url = url_part_1 + str(latitude_a) + url_part_2 + str(longitude_a) + url_part_3
        url_list += [url]
        
    
    longitude_a = 37.067043 # возвращаю точку к исходной долготе
    latitude_a -= 0.01003 # перемещаю точку по широте на юг

In [ ]:
# проверяю длину списка

len(url_list)

Собираю данные и ссылки на страницы клиник на Zoon

In [ ]:
# список для данных
data = []


for map_url in tqdm(url_list[240:301]): # цикл для перебора ссылок (точек входа на карте)
    
    sleep(3)
    driver.get(map_url)
    driver.implicitly_wait(5)
    
    # код на случай, если вылезет капча
    if driver.find_element_by_class_name('simple-text').text == 'Мы проверяем, что вы не робот, через несколько секунд вы будете перенаправлены на сайт.':
        print('Stop')
        break
    
    # нажимаю 8 раз кнопку "показать следующие": zoon отдает, максимум, 240 клиник в одном окне
    for i in range(10):
        sleep(2)
        
        try:
            driver.find_element_by_class_name('js-next-page').click()
        
        except:
            break
            
    object_blocks = driver.find_elements_by_class_name('service-item')
    
    if len(object_blocks) > 0: # сбор данных, если объекты есть на карте
        
        # цикл для выгрузки данных
        for object_page in object_blocks:
            
            topic = object_page.find_element_by_class_name('H3').text # название
            
            try:
                object_type = object_page.find_element_by_class_name('address-info-features').text # тип объекта
                
            except:
                object_type = 'не указан'
                
            longitude = object_page.get_attribute('data-lon') # широта
            latitude = object_page.get_attribute('data-lat') # долгота
            
            address = object_page.find_element_by_tag_name('address').text # адрес
            clinic_page = object_page.find_element_by_tag_name('a').get_attribute('href') # страница объекта
            
            data.append([topic, object_type, longitude, latitude, address, clinic_page])

In [ ]:
# записываю данные в датафрейм

head_list = ['topic', 'object_type', 'longitude', 'latitude', 'address', 'clinic_page']
work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.shape

In [ ]:
work.sample(3)

In [ ]:
# Проверяю дубликаты. Дубликатов должно быть много из-за близости точек

work[work.duplicated(keep=False)].shape

In [ ]:
# Удаляю дубликаты

work.drop_duplicates(inplace=True)

In [ ]:
work.shape

In [ ]:
work.sample(3)

In [ ]:
# Записываю в файл

work.to_excel('C:/Data/stom_Zoom_urls_1.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.quit()